In [ ]:
bert2_path='outputs/2024-04-15/02-32-46-059563/checkpoints/val/genomic_structure_corr.ckpt'
hyena_path='outputs/2024-04-15/08-34-24-880261/checkpoints/val/genomic_structure_corr.ckpt'
orca_path='outputs/2024-04-15/09-19-56-380804/checkpoints/val/genomic_structure_corr.ckpt'
mamba_path='outputs/2024-04-20/03-18-38-491508/checkpoints/val/genomic_structure_corr.ckpt'
import sys

from selene_utils2 import *
from matplotlib import pyplot as plt
batch_size=5
max_length=6000
dest_path='data/genomic_structure/resources/resources'
t = Genomic2DFeatures(
            [dest_path + "/4DNFI9GMP2J8.rebinned.mcool::/resolutions/1000"],
            ["r1000"],
            (int(max_length/1000), int(max_length/1000)),
            cg=True,
        )

sampler = RandomPositionsSamplerHiC(
            reference_sequence=MemmapGenome(
                input_path=dest_path + "/Homo_sapiens.GRCh38.dna.primary_assembly.fa",
                memmapfile=dest_path + "/Homo_sapiens.GRCh38.dna.primary_assembly.fa.mmap",
                blacklist_regions="hg38",
            ),
            target=t,
            # target_1d=MultibinGenomicFeatures(
            #     self.dest_path + "/h1esc/h1esc.hg38.bed.sorted.gz",
            #     np.loadtxt(self.dest_path + "/h1esc/h1esc.hg38.bed.sorted.features", str),
            #     1000,
            #     1000,
            #     (32, 10),
            #     mode="any",
            # ),
            features=["r1000"],
            test_holdout=["chr9", "chr10"],
            validation_holdout=["chr8"],
            sequence_length=6000,
            position_resolution=1000,
            random_shift=100,
            random_strand=False,
            cross_chromosome=False,
        )
sampler.mode = "validate"
for i in range(10):
    sampled_data = sampler.sample(batch_size=batch_size)

    while np.isnan(sampled_data[1]).any():
                sampled_data = sampler.sample(batch_size=batch_size)

# print(sampled_data)
plt.imshow(sampled_data[1][0, :, :],cmap = 'Spectral_r')   
plt.colorbar()


In [ ]:
sequence=sampled_data[0]
target=sampled_data[1]
target=torch.from_numpy(target).float().cuda()
sequence_b=np.zeros((sequence.shape[0],sequence.shape[1]))
mask=np.where(sequence==0.25)
sequence_b=sequence.argmax(axis=2)
sequence_b[[mask[0],mask[1]]]=4
sequence=sequence_b
map_to_genomic_sequence = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: 'N'}
#map sequence to genome 
sequence = "".join(map(lambda x: map_to_genomic_sequence[x], sequence.flatten().tolist()))
#reshape list
# sequence = np.array(list(sequence)).reshape(batch_size,-1)
from scipy.stats import pearsonr
def genomic_structure_corr(logit,y,ignore_index=-100):


    pred=logit
    target=y
    normmat_bydist = np.exp(
        np.load("orca/resources/resources/4DNFI643OYP9.rebinned.mcool.expected.res1000.npy")        

    )[:10]
    normmat = normmat_bydist[np.abs(np.arange(6)[:, None] - np.arange(6)[None, :])]
    normmat_r = torch.from_numpy(normmat).float().cuda()
    eps = torch.min(normmat_r)
    target = torch.log(((target + eps) / (normmat_r + eps)))
    corr=[]
    #convert to numpy
    target=target.cpu().numpy().reshape((pred.shape[0], -1))
    pred=pred.detach().cpu().numpy().reshape((pred.shape[0], -1))
    for j in range(pred.shape[0]):
            if np.mean(np.isnan(target[j, :])) < 0.7:
                corr.append(
                    pearsonr(
                        pred[j, ~np.isnan(target[j, :])],
                        target[j, ~np.isnan(target[j, :])],
                    )[0]
                )
            else:
                corr.append(np.nan)
    corr=np.nanmean(corr)
    return corr
def genomic_structure_mse(logit,y,ignore_index=-100):

    pred=logit
    target=y
    if torch.isnan(target).all():
        target=torch.rand_like(target)

    normmat_bydist = np.exp(
        np.load("orca/resources/resources/4DNFI643OYP9.rebinned.mcool.expected.res1000.npy")

    )[:8]
    normmat = normmat_bydist[np.abs(np.arange(6)[:, None] - np.arange(6)[None, :])]
    normmat_r = torch.from_numpy(normmat).float().cuda()
    eps = torch.min(normmat_r)
    target = torch.nanmean(
                torch.nanmean(torch.reshape(target, (target.shape[0], 6, 1, 6, 1)), axis=4),
                axis=2,
            )
    target_r = torch.log(((target + eps) / (normmat_r + eps)))
    target_cuda = target_r

    loss = (
        (
            pred[~torch.isnan(target)]
            - target_cuda[~torch.isnan(target)]
        )
        ** 2
    ).mean()



    return loss

In [ ]:
#import autotokenizer
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn

with torch.no_grad():
    hyena_tokenizer=AutoTokenizer.from_pretrained("weight/hyenadna/hyenadna-large-1m-seqlen",trust_remote_code=True)
    hyena_model=AutoModel.from_pretrained("weight/hyenadna/hyenadna-large-1m-seqlen",trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('outputs/2024-04-14/01-11-41-629746/checkpoints/last.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.length_transform_2."
        )

    hyena_decoder = nn.Linear(6000,36,bias=False).to('cuda')
    #edit key name in hyena_decoder
    state_dict=hyena_decoder.state_dict()
    
    hyena_model.load_state_dict(checkpoint,strict=False)
    hyena_decoder.load_state_dict(checkpoint,strict=False)
    hyena_model.eval()
    hyena_decoder.eval()

    sequence_encoded=hyena_tokenizer(sequence,
                        add_special_tokens= False,  # this is what controls adding eos
                        padding="max_length",
                        max_length=max_length*batch_size,
                        truncation=True,
                    )


    sequence_tensor=torch.tensor(sequence_encoded['input_ids']).to('cuda')
    sequence_tensor=torch.reshape(sequence_tensor,(batch_size,max_length))
    hidden_states=hyena_model(sequence_tensor)['last_hidden_state']
    output=hidden_states[..., -6000:, :]
    out1 = output[:,:,0]
    out1=hyena_decoder(out1)
    out1=out1.reshape(out1.shape[0],6,6)
    mat_hyena=out1+out1.transpose(1,2)

    #calculate the 
    corr=genomic_structure_corr(mat_hyena,target)
    print(corr)
    mse=genomic_structure_mse(mat_hyena,target)
    print(mse)
from matplotlib import pyplot as plt
def figshow(x, np=False):
        if np:
            plt.imshow(x.squeeze())
        else:
            plt.imshow(x.squeeze().cpu().detach().numpy())
        #remove boudary
        plt.axis('off')
        plt.show()
#plot the target and predicted genomic structure
# figure,(axes1, axes2, axes3, axes4, axes5)=plt.subplots(1, 5,figsize=(20,5))
# axes1.imshow(target[0].squeeze().cpu().detach().numpy())
# axes1.axis('off')
# axes1

plt.imshow(mat_hyena[0].cpu().detach().numpy(),cmap = 'Spectral_r')
plt.colorbar()

In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn
with torch.no_grad():
    state_dict='weight/dnabert2'
    bert2_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    bert2_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('outputs/2024-04-15/02-32-46-059563/checkpoints/last.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.length_transform_2."
        )

    bert2_decoder = nn.Linear(6000,36,bias=False).to('cuda')
    #edit key name in hyena_decoder
    
    bert2_model.load_state_dict(checkpoint,strict=False)
    bert2_decoder.load_state_dict(checkpoint,strict=False)
    bert2_model.eval()
    bert2_decoder.eval()

    sequence_encoded=bert2_tokenizer(sequence,
                        add_special_tokens= False,  # this is what controls adding eos
                        padding="max_length",
                        max_length=max_length*batch_size,
                        truncation=True,
                    )


    sequence_tensor=torch.tensor(sequence_encoded['input_ids']).to('cuda')
    sequence_tensor=torch.reshape(sequence_tensor,(batch_size,max_length))
    hidden_states=bert2_model(input_ids=sequence_tensor,output_hidden_states=True,)[0]
    output=hidden_states[..., -6000:, :]
    out1 = output[:,:,0]
    out1=bert2_decoder(out1)
    out1=out1.reshape(out1.shape[0],6,6)
    mat_bert2=out1+out1.transpose(1,2)

    #calculate the 
    corr=genomic_structure_corr(mat_bert2,target)
    print(corr)
    mse=genomic_structure_mse(mat_bert2,target)
    print(mse)

In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn
with torch.no_grad():
    state_dict='weight/mamba'
    mamba_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    mamba_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('outputs/2024-04-20/03-18-38-491508/checkpoints/last.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.length_transform_2."
        )

    mamba_decoder = nn.Linear(6000,36,bias=False).to('cuda')
    #edit key name in hyena_decoder
    
    mamba_model.load_state_dict(checkpoint,strict=False)
    mamba_decoder.load_state_dict(checkpoint,strict=False)
    mamba_model.eval()
    mamba_decoder.eval()

    sequence_encoded=mamba_tokenizer(sequence,
                        add_special_tokens= False,  # this is what controls adding eos
                        padding="max_length",
                        max_length=max_length*batch_size,
                        truncation=True,
                    )


    sequence_tensor=torch.tensor(sequence_encoded['input_ids']).to('cuda')
    sequence_tensor=torch.reshape(sequence_tensor,(batch_size,max_length))
    hidden_states=mamba_model(input_ids=sequence_tensor,output_hidden_states=True,)[0]
    output=hidden_states[..., -6000:, :]
    out1 = output[:,:,0]
    out1=mamba_decoder(out1)
    out1=out1.reshape(out1.shape[0],6,6)
    mat_mamba=out1+out1.transpose(1,2)

    #calculate the 
    corr=genomic_structure_corr(mat_mamba,target)
    print(corr)

In [ ]:
import sys

from src.models.sequence.orca import Orca
orca_model=Orca().to('cuda')
seq_ids =torch.from_numpy(sampled_data[0]).squeeze(0)
seq_ids = seq_ids.float().to('cuda')
target = torch.from_numpy(sampled_data[1]).squeeze(0)
target = target.float().to('cuda')
parameter=torch.load('outputs/2024-04-15/09-19-56-380804/checkpoints/last.ckpt')['state_dict']
checkpoint=torch.load('outputs/2024-04-15/09-19-56-380804/checkpoints/last.ckpt')['state_dict']
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.length_transform_2."
        )
orca_model.load_state_dict(checkpoint,strict=False)
out_orca=orca_model(seq_ids)[0]
corr=genomic_structure_corr(out_orca,target)
print(corr)

In [ ]:
#plot nxn image
rows = ['GT','Orca','Caduceus','Hyena-DNA','DNABERT2']
out_collection=[]
normmat_bydist = np.exp(
                np.load("orca/resources/resources/4DNFI643OYP9.rebinned.mcool.expected.res1000.npy")
            )[:6]
normmat = normmat_bydist[np.abs(np.arange(6)[:, None] - np.arange(6)[None, :])]
normmat_r = torch.from_numpy(normmat).float().cuda()
eps = torch.min(normmat_r)
target_r=torch.log(((target + eps) / (normmat_r + eps)))
out_collection.append(target_r.cpu().detach().numpy())
out_collection.append(out_orca.cpu().detach().numpy())
out_collection.append(mat_mamba.cpu().detach().numpy())
out_collection.append(mat_hyena.cpu().detach().numpy())
out_collection.append(mat_bert2.cpu().detach().numpy())
_min, _max = np.amin(out_collection), np.amax(out_collection)
fig,axes = plt.subplots(5,5)
for ax, row in zip(axes[:,0], rows):
    ax2 = ax.twinx()
    ax2.yaxis.set_label_position('left')
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    #set ticks invisible
    ax2.tick_params(axis='both', which='both', length=0)
    ax2.set_yticks([])
    ax2.set_ylabel(row, rotation=0, size='large',
                   ha='right', va='center')
for i in range(5):
    for j in range(5):
        ax_n=axes[i,j].imshow(out_collection[i][j], vmin = _min, vmax = _max)
        axes[i,j].axis('off')
        axes[i,j].autoscale(False)
fig.colorbar(ax_n,ax=axes,orientation='vertical')

plt.suptitle('Genomic Structure Prediction',fontsize=15,y=0.95,fontweight='bold')
plt.show()
        #show value bar
